<a href="https://colab.research.google.com/github/sandeep16064/Named-Entity-Recognition-NER-Papers/blob/master/model%20of%2004_NER_using_spaCy_CoNLL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training and Evaluating an NER model with spaCy on the CoNLL dataset

In this notebook, we will take a look at using spaCy commandline to train and evaluate a NER model. We will also compare it with the pretrained NER model in spacy. 

Note: we will create multiple folders during this experiment:
spacyNER_data 

## Step 1: Converting data to json structures so it can be used by Spacy

In [1]:
import os

In [3]:
# upload train.txt, test.txt, valid.txt from Data/conll2003/en
try:
    from google.colab import files
    uploaded = files.upload()
except ModuleNotFoundError:
    print('Not using colab')

Saving dev.txt to dev.txt
Saving test.txt to test.txt
Saving train.txt to train (1).txt


In [6]:
#Read the CONLL data from conll2003 folder, and store the formatted data into a folder spacyNER_data

# !mkdir spacyNER_data
os.mkdir('spacyNER_data')
        
#the above lines create folder if it doesn't exist. If it does, the output shows a message that it
#already exists and cannot be created again
try:
    import google.colab 
    !python -m spacy convert "train.txt" spacyNER_data -c ner
    !python -m spacy convert "test.txt" spacyNER_data -c ner
    !python -m spacy convert "dev.txt" spacyNER_data -c ner
except ModuleNotFoundError:
    !python -m spacy convert "Data/conll2003/en/train.txt" spacyNER_data -c ner
    !python -m spacy convert "Data/conll2003/en/test.txt" spacyNER_data -c ner
    !python -m spacy convert "Data/conll2003/en/dev.txt" spacyNER_data -c ner

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (14987 documents): spacyNER_data/train.json
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (3466 documents): spacyNER_data/test.json
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (3466 documents): spacyNER_data/dev.json


#### For example, the data before and after running spacy's convert program looks as follows.

In [7]:
try:
    import google.colab
    !echo "BEFORE : (train.txt)"
    !head "train.txt" -n 11 | tail -n 9
except ModuleNotFoundError:
    print("BEFORE : (Data/conll2003/en/train.txt)")
    file = open("Data/conll2003/en/train.txt")
    content = file.readlines()
    print(*content[1:11])

BEFORE : (train.txt)
EU NNP B-NP B-ORG
rejects VBZ B-VP O
German JJ B-NP B-MISC
call NN I-NP O
to TO B-VP O
boycott VB I-VP O
British JJ B-NP B-MISC
lamb NN I-NP O
. . O O


In [8]:
try:
    import google.colab
    !echo "AFTER : (spacyNER_data/train.json)"
    !head "spacyNER_data/train.json" -n 77 | tail -n 58
except ModuleNotFoundError:
    print("AFTER : (spacyNER_data/train.json)")
    f = open('spacyNER_data/train.json')
    content = f.readlines()
    print(*content[19:77])

AFTER : (spacyNER_data/train.json)
  {
    "id":1,
    "paragraphs":[
      {
        "sentences":[
          {
            "tokens":[
              {
                "orth":"EU",
                "tag":"NNP",
                "ner":"U-ORG"
              },
              {
                "orth":"rejects",
                "tag":"VBZ",
                "ner":"O"
              },
              {
                "orth":"German",
                "tag":"JJ",
                "ner":"U-MISC"
              },
              {
                "orth":"call",
                "tag":"NN",
                "ner":"O"
              },
              {
                "orth":"to",
                "tag":"TO",
                "ner":"O"
              },
              {
                "orth":"boycott",
                "tag":"VB",
                "ner":"O"
              },
              {
                "orth":"British",
                "tag":"JJ",
                "ner":"U-MISC"
              },
              {


## Training the NER model with Spacy (CLI)

All the commandline options can be seen at: https://spacy.io/api/cli#train
We are training using the train program in spacy, for English (en), and the results are stored in a folder 
called "model" (created while training). Our training file is in "spacyNER_data/train.json" and the validation file is at: "spacyNER_data/valid.json". 

-G stands for gpu option.
-p stands for pipeline, and it should be followed by a comma separated set of options - in this case, a tagger and an NER are being trained simultaneously

In [10]:
!python -m spacy train en model spacyNER_data/train.json spacyNER_data/dev.json -G -p tagger,ner

✔ Created output directory: model
Training pipeline: ['tagger', 'ner']
Starting with blank model 'en'
Counting training words (limit=0)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W022] Training a new part-of-speech tagger using a model with no lemmatization rules or data. This means that the trained model may not be able to lemmatize correctly. If this is intentional or the language you're using doesn't have lemmatization data, you can ignore this warning by setting SPACY_WARNING_IGNORE=W022. If this is surprising, make sure you have the spacy-lookups-data package installed.
  "__main__", mod_spec)

Itn  Tag Loss    Tag %    NER Loss   NER P   NER R   NER F   Token %  CPU WPS
---  ---------  --------  ---------  ------  ------  ------  -------  -------
  1  31411.855    94.143  16692.765  83.458  82.868  83.162  100.000    12182
  2  16916.010    94.819   7588.607  86.702  86.133  86.416  100.000    12355
  3  13723.936    95.130   5222.184  88.040  87.462  87.750  100.000    12042

Notice how the performance improves with each iteration!
## Evaluating the model with test data set (`spacyNER_data/test.json`)

### On Trained model (`model/model-best`)

In [16]:
#create a folder to store the output and visualizations. 
# !mkdir result
os.mkdir('result')
!python -m spacy evaluate model/model-best spacyNER_data/test.json -dp result
# !python -m spacy evaluate model/model-final data/test.txt.json -dp result


================================== Results ==================================

Time      4.29 s
Words     51578 
Words/s   12024 
TOK       100.00
POS       95.59 
UAS       0.00  
LAS       0.00  
NER P     88.75 
NER R     88.15 
NER F     88.45 
Textcat   0.00  

/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure t

a Visualization of the entity tagged test data can be seen in result/entities.html folder. 

### On spacy's Pretrained NER model (`en_core_web_sm`)

In [12]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [13]:
# !mkdir pretrained_result
os.mkdir('pretrained_result')
!python -m spacy evaluate en_core_web_sm spacyNER_data/test.json -dp pretrained_result


================================== Results ==================================

Time      7.77 s
Words     51578 
Words/s   6637  
TOK       100.00
POS       87.09 
UAS       0.00  
LAS       0.00  
NER P     5.15  
NER R     7.17  
NER F     5.99  
Textcat   0.00  

/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)
✔ Generated 25 parses as HTML
pretrained_result


a Visualization of the entity tagged test data can be seen in pretrained_result/entities.html folder. 